In [2]:
import pandas as pd

df1 = pd.read_excel('online_retail_II.xlsx', sheet_name=0, index_col=0)
df1.head()

,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
Invoice,,,,,,,
489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [3]:
df = df1[df1['Country'] != 'United Kingdom']
df.describe()

,Quantity,Price,Customer ID
count,39609.000000,39609.000000,38111.000000
mean,25.322401,6.471909,13376.564981
std,249.749198,93.092996,1153.948898
min,-7128.000000,0.000000,12347.000000
25%,4.000000,1.250000,12519.000000
50%,10.000000,1.950000,12705.000000
75%,16.000000,3.750000,14564.000000
max,19152.000000,6958.170000,18140.000000


In [4]:
stocks = df[['StockCode', 'Description']]
stocks = stocks.dropna()
stocks = stocks.drop_duplicates()
stocks.to_csv('stocks.csv', index=False)

In [5]:
import numpy as np

customers = df[['Customer ID', 'Country']]
customers = customers.dropna()
customers = customers.drop_duplicates()
customers['Customer ID'] = customers['Customer ID'].astype(np.int64)
customers.to_csv('customers.csv', index=False)

In [6]:
invoices = df[['StockCode', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID']]
invoices = invoices.dropna()
invoices['Customer ID'] = invoices['Customer ID'].astype(np.int64)
invoices.to_csv('invoices.csv')

In [9]:
import mysql.connector

conn = mysql.connector.connect(
    host='localhost',
    port=3306,
    user='root',
    password='password',
    database='lfit_kadai'
)

cur = conn.cursor(dictionary=True)

In [10]:
import csv

with open('stocks.csv', 'r') as f:
    reader = csv.reader(f)
    first = True
    for line in reader:
        if first:
            first = False
            continue
        cur.execute('INSERT INTO analysis_stock VALUES (NULL, %s, %s)', (line[0], line[1],))
        
conn.commit()

In [11]:
with open('customers.csv', 'r') as f:
    reader = csv.reader(f)
    first = True
    for line in reader:
        if first:
            first = False
            continue
        cur.execute('INSERT INTO analysis_customer VALUES (NULL, %s, %s)', (line[0], line[1],))
        
conn.commit()

In [12]:
with open('invoices.csv', 'r') as f:
    reader = csv.reader(f)
    first = True
    for line in reader:
        if first:
            first = False
            continue
        cur.execute('INSERT INTO analysis_invoiceline VALUES (NULL, %s, %s, %s, %s, %s, %s, %s, %s)', (
            line[0], 
            line[1], 
            line[2], 
            line[3], 
            line[4], 
            line[5], 
            line[0].startswith('C'), 
            int(line[2]) * float(line[4])
        ))
        
conn.commit()